In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import col
from pyspark.sql.functions import format_number

from pyspark.sql.functions import round

## SPARK SESSION

In [2]:
spark = SparkSession.builder.appName("sparkhive").enableHiveSupport().getOrCreate()

## Data Extraction from Hive

In [6]:
df = spark.sql("SELECT * FROM fraud_project.fraud_full_load_external")

In [7]:
#Adding an incremental id to keep tranck of the transactions
df = df.withColumn("id", monotonically_increasing_id())

In [8]:
#Visualizing the features that we will be using in our model
selected_columns = ["id","transaction_type","amount", "oldbalanceorg", "newbalanceorig", "oldbalancedest", "newbalancedest", "isfraud"]
selected_df = df.select(*selected_columns)
selected_df.show()

+---+----------------+---------+-------------+--------------+--------------+--------------+-------+
| id|transaction_type|   amount|oldbalanceorg|newbalanceorig|oldbalancedest|newbalancedest|isfraud|
+---+----------------+---------+-------------+--------------+--------------+--------------+-------+
|  0|            type|   amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|
|  1|         PAYMENT|  9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|
|  2|         PAYMENT|  1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|
|  3|        TRANSFER|    181.0|        181.0|           0.0|           0.0|           0.0|      1|
|  4|        CASH_OUT|    181.0|        181.0|           0.0|       21182.0|           0.0|      1|
|  5|         PAYMENT| 11668.14|      41554.0|      29885.86|           0.0|           0.0|      0|
|  6|         PAYMENT|  7817.71|      53860.0|      46042.29|           0.0|           0.0|      0|


In [ ]:
#num_columns = len(df.columns)


## Exploratory Data Analysis (EDA)

#Statistical Analysis
#Plot some feature distributions
#Correlation Matrix

In [9]:
#Dropping the columns that we won't be using
columns_to_drop=["step", "nameOrig", "nameDest", "isFlaggedFraud", "row_id", "id"]
df_new=df.drop(*columns_to_drop)
#df_new.show()

# Specify the value you want to filter out
value_to_remove = "1.0E7" 

# Remove rows where the "account" column has the specified value
filtered_df = df_new.filter(df_new.amount != value_to_remove)

# Calculate summary statistics with one decimal place
summary = filtered_df.summary()

summary = summary.select(
    summary['summary'],
    #format_number(summary['transaction_type'].cast('double'), 1).alias('transaction_type'),
    format_number(summary['amount'].cast('double'), 1).alias('amount'),
    format_number(summary['oldbalanceorg'].cast('double'), 1).alias('oldbalanceorg'),
    format_number(summary['newbalanceorig'].cast('double'), 1).alias('newbalanceorig'),
    format_number(summary['oldbalancedest'].cast('double'), 1).alias('oldbalancedest'),
    format_number(summary['newbalancedest'].cast('double'), 1).alias('newbalancedest'),
    #format_number(summary['isfraud'].cast('double'), 1).alias('isfraud')
)

summary.show()

# Round the "amount" column to a specified number of decimal places (e.g., 2)
#rounded_df = df.withColumn("rounded_amount", round("amount", 2))


+-------+---------+-------------+--------------+--------------+--------------+
|summary|   amount|oldbalanceorg|newbalanceorig|oldbalancedest|newbalancedest|
+-------+---------+-------------+--------------+--------------+--------------+
|  count|  5,070.0|      5,070.0|       5,070.0|       5,070.0|       5,070.0|
|   mean| 96,491.7|  1,024,118.8|   1,049,475.2|     906,199.2|   1,050,553.7|
| stddev|225,373.4|  2,279,156.8|   2,330,085.8|   2,526,749.8|   2,903,139.7|
|    min| 10,006.7|          0.0|           0.0|           0.0|           0.0|
|    25%|  3,719.9|        574.0|           0.0|           0.0|           0.0|
|    50%| 10,728.8|     26,165.3|      17,790.1|           0.0|           0.0|
|    75%|107,292.2|    281,826.4|     281,759.5|     353,797.3|     265,092.4|
|    max|     null|         null|          null|          null|          null|
+-------+---------+-------------+--------------+--------------+--------------+



In [24]:
# Check if the specified column has null values
null_count = filtered_df.filter(col('amount').isNull()).count()

# Print the result
if null_count > 0:
    print(f"The column has null values.")
    print(null_count)
else:
    print(f"The column does not have any null values.")
    print(null_count)

The column does not have any null values.
0


In [33]:
# Specify the column you want to analyze
column_to_check = "newbalancedest"

# Check if the specified column has values that are not float or integer
non_float_or_integer_count = filtered_df.filter(
    col(column_to_check).cast("double").isNull() &
    (col(column_to_check).cast("string") != "")
).count()

# Print the result
if non_float_or_integer_count > 0:
    print(f"The column '{column_to_check}' has {non_float_or_integer_count} values that are not float or integer.")
else:
    print(f"The column '{column_to_check}' does not have any values that are not float or integer.")

The column 'newbalancedest' has 2 values that are not float or integer.


In [ ]:
# Calculate summary statistics with one decimal place
summary = filtered_df.summary()

summary = summary.select(
    summary['summary'],
    #format_number(summary['transaction_type'].cast('double'), 1).alias('transaction_type'),
    format_number(summary['amount'].cast('double'), 1).alias('amount'),
    format_number(summary['oldbalanceorg'].cast('double'), 1).alias('oldbalanceorg'),
    format_number(summary['newbalanceorig'].cast('double'), 1).alias('newbalanceorig'),
    format_number(summary['oldbalancedest'].cast('double'), 1).alias('oldbalancedest'),
    format_number(summary['newbalancedest'].cast('double'), 1).alias('newbalancedest'),
    #format_number(summary['isfraud'].cast('double'), 1).alias('isfraud')
)

summary.show()

In [10]:
# Check the data types of each column
column_data_types = filtered_df.dtypes

# Print the data types
for column, data_type in column_data_types:
    print(f"Column '{column}' has data type '{data_type}'")

Column 'transaction_type' has data type 'string'
Column 'amount' has data type 'string'
Column 'oldbalanceorg' has data type 'string'
Column 'newbalanceorig' has data type 'string'
Column 'oldbalancedest' has data type 'string'
Column 'newbalancedest' has data type 'string'
Column 'isfraud' has data type 'string'


## Feature Engineering

#Null Values
#Imputation Techniques 
#Scale the data
#One Hot Encoding
#Techniques to balance the data (Oversampling and Undersampling Techniques)

## MODELING

#Split the dataset into training and testing
#Logistic regression, 2 Ensemble techniques (Randon Forest and XGBoost)


## EVALUATION

#Accuracy
#Recall
#Precision
#F1 Score
#AUC-ROC